In [1]:
from openai import OpenAI
import re

In [2]:
client = OpenAI(api_key="FAKE", base_url="https://openai.jabirproject.org/v1")

## Testing the API

In [3]:
completion = client.chat.completions.create(
    model = "jabir-400b",
    messages = [
        {
            "role" : "user",
            "content" : "What is the answer to life, universe and everything?"
        }
    ]
)

In [4]:
completion.choices[0].message.content

'The answer to life, the universe, and everything is famously noted as "42," according to Douglas Adams\' science fiction series "The Hitchhiker\'s Guide to the Galaxy." However, the question itself remains unknown, leaving the answer somewhat humorous and enigmatic.'

## Making a simple Agent

In [5]:
class Agent:
    
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role" : "system", "content" : system})
    
    def __call__(self, message):
        self.messages.append({"role" : "user", "content" : message})
        result = self.execute()
        self.messages.append({"role" : "assistant", "content" : result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            model = "jabir-400b",
            messages = self.messages,
            temperature = 0.0
        )
        
        return completion.choices[0].message.content
    

## Testing the agent

In [6]:
sample_agent = Agent("You are a helpful assistant")

In [7]:
sample_agent("What is 1+1")

'1 + 1 equals 2.'

In [8]:
sample_agent("Who made you?")

'I was developed by Muhammadreza Haghiri as part of the Jabir Project.'

In [9]:
sample_agent.messages

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': 'What is 1+1'},
 {'role': 'assistant', 'content': '1 + 1 equals 2.'},
 {'role': 'user', 'content': 'Who made you?'},
 {'role': 'assistant',
  'content': 'I was developed by Muhammadreza Haghiri as part of the Jabir Project.'}]

## Actions

In [10]:
import math

def nth_occurance_of_fibonacci(n):
    sqrt_5 = math.sqrt(5)
    phi = (1 + sqrt_5) / 2  # Golden ratio
    psi = (1 - sqrt_5) / 2  # Conjugate of the golden ratio
    
    # Compute the Fibonacci number using the formula
    F_n = (1 / sqrt_5) * (phi**n - psi**n)
    return round(F_n)

In [11]:
nth_occurance_of_fibonacci(15)

610

In [12]:
nth_occurance_of_fibonacci(5)

5

In [14]:
nth_occurance_of_fibonacci(1000)

43466557686938914862637500386755014010958388901725051132915256476112292920052539720295234060457458057800732025086130975998716977051839168242483814062805283311821051327273518050882075662659534523370463746326528

## Making an agent for doing some tasks

In [15]:
action_re = re.compile("^Action: (\w+): (.*)$")

In [16]:
actions = {
    "nth_occurance_of_fibonacci" : nth_occurance_of_fibonacci
}

In [17]:
system_prompt = """
You are an action agent called Smith. You have the task to analyze user input and suggest the action, based on the list of available actions.
Then, when the result of the action is sent back to you, you have to humanize the output. 

List of available tasks:
- nth_occurance_of_fibonacci

Example of the user's interaction with you:

User: What is the 15th occurance of Fibbonacci? 
Assistant: Action: nth_occurance_of_fibonaci: 15
User: The result of the action was ACTION_RESULR
Assistant: HUMANIZED_VERSION_OF_WHAT_USER_PROVIDED
"""

In [18]:
fibonacci_agent = Agent(system_prompt)

In [19]:
action = fibonacci_agent("What is 1001th occurance of fibonacci series?")

In [20]:
print(action)

Action: nth_occurance_of_fibonacci: 1001


In [21]:
def action_processing(action, actions):
    
    action = action_re.match(action)
    action = action[0].split("Action:")
    action = action[1]
    action = action.split(':')
    
    action_function = action[0].split(' ')[1]
    action_input = action[1].split(' ')[1]
    
    if action_function == "nth_occurance_of_fibonacci":
        result = actions[action_function](int(action_input))
    
    return result
    

In [22]:
result = action_processing(action, actions)

In [23]:
result

70330367711425174290694829277217085237890517319611544836492248291277937018392669473182883449499700079838702138514693954865215540324463972269041336236408103803463044485157310035600578886573282113950100689518592

In [24]:
fibonacci_agent(f"The result of the action was {result}")

'The 1001st occurrence of the Fibonacci series is an incredibly large number: 70330367711425174290694829277217085237890517319611544836492248291277937018392669473182883449499700079838702138514693954865215540324463972269041336236408103803463044485157310035600578886573282113950100689518592. This highlights how quickly Fibonacci numbers can grow!'